In [17]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('results') if isfile(join('results', f))]

In [29]:
result_files = []
for file in onlyfiles:
    try:
        if file.count('_') >= 3:
            method = [m for m in ['v125', 'v49'] if m in file][0]
            method = method.upper() if method in ['macbf', 'ddpg'] else 'CAM'
            environment = [env for env in ['dubins_car', 'drone'] if env in file][0]
            density = int(file.split('_')[-1].replace('.pkl', ''))
            mode = [m for m in ['mixed'] if m in file][0]
            print(f'{file},{environment},{mode},{method},{density}')
            result_files.append([file,environment,mode,method,density])
        else:
            continue
    except:
        continue

drone_v49_mixed_8.pkl,drone,mixed,CAM,8
drone_v49_mixed_32.pkl,drone,mixed,CAM,32
drone_v49_mixed_16.pkl,drone,mixed,CAM,16
dubins_car_v125_mixed_128.pkl,dubins_car,mixed,CAM,128
dubins_car_v125_mixed_1.pkl,dubins_car,mixed,CAM,1
dubins_car_v125_mixed_16.pkl,dubins_car,mixed,CAM,16
drone_v49_mixed_256.pkl,drone,mixed,CAM,256
dubins_car_v125_mixed_8.pkl,dubins_car,mixed,CAM,8
drone_v49_mixed_64.pkl,drone,mixed,CAM,64
drone_v49_mixed_512.pkl,drone,mixed,CAM,512
dubins_car_v125_mixed_64.pkl,dubins_car,mixed,CAM,64
dubins_car_v125_mixed_256.pkl,dubins_car,mixed,CAM,256
dubins_car_v125_mixed_32.pkl,dubins_car,mixed,CAM,32
drone_v49_mixed_1.pkl,drone,mixed,CAM,1
drone_v49_mixed_3.pkl,drone,mixed,CAM,3
dubins_car_v125_mixed_3.pkl,dubins_car,mixed,CAM,3
dubins_car_v125_mixed_512.pkl,dubins_car,mixed,CAM,512
drone_v49_mixed_128.pkl,drone,mixed,CAM,128


In [3]:
def get_map_size(mode, density):
    if mode=='static':
        size = 4
    if mode=='dynamic':
        size = int((density*16)**0.5)
    if mode=='mixed':
        size = 32
    return size

def get_agent_num(mode, density):
    if mode=='static':
        num_agents = 3
    if mode=='dynamic':
        num_agents = density
    if mode=='mixed':
        num_agents = density
    return num_agents

In [70]:
# save the environment,dataset,method,density,collision_per_env,collision_per_agent,success_rate,path_length

def get_reward(num_collides, num_success, path_length, agent_num):
    return (num_collides*(-1) + num_success*10)/agent_num + -1e-2*path_length

def get_safe_rate(num_collides, path_length, agent_num):
    return 1-(num_collides/(path_length*agent_num))

In [30]:
environment = 'dubins_car'

In [71]:
import seaborn as sns
sns.set_style("dark")
total_results = sns.load_dataset("/results", data_home='./results')

In [31]:
import pickle as pkl
import numpy as np

result_files = [r for r in result_files if r[1]==environment]
for file,environment,mode,method,density in result_files:
    
    if environment=='drone':
        from drone_v34 import *
    elif environment=='dubins_car':
        from v109 import *    
    
    with open(f'dataset/{environment}_{mode}_{density}.pkl', 'rb') as f:
        valid_dataset = pkl.load(f)
        
    with open(f'results/{file}', 'rb') as f:
        result_dict = pkl.load(f)
        paths = result_dict['paths']
    
    map_size = get_map_size(mode, density)
    num_agents = get_agent_num(mode, density)
    env = create_env(num_agents=num_agents, size=map_size, density=0, min_dist=0)

    # reproduce the trajectory
    num_subgraphs = []
    for param, path in zip(valid_dataset[:10], paths[:10]):
        env.world.obstacles, env.world.agent_goals, env.world.agents = deepcopy(param)
        num_subgraph = 0
        for timestep in path:
            env.world.agents = timestep
            num_subgraph += len(list(env._get_obs_random_k(**OBS_CONFIG_DECOMPOSE)))
        num_subgraphs.append(num_subgraph/len(path))
    print(f'{environment},{mode},{method},{density},{np.mean(num_subgraphs)},{len(path)}')

dubins_car,mixed,CAM,128,11.529945272360123,257
dubins_car,mixed,CAM,1,1.3312529825741986,101
dubins_car,mixed,CAM,16,3.1608249178767114,113
dubins_car,mixed,CAM,8,2.455574290250312,113
dubins_car,mixed,CAM,64,5.89744962663882,129
dubins_car,mixed,CAM,256,22.177645400600298,257
dubins_car,mixed,CAM,32,3.966920241353433,115
dubins_car,mixed,CAM,3,1.777849994364886,101
dubins_car,mixed,CAM,512,43.55136186770428,257


In [6]:
%debug

> /tmp/ipykernel_524194/2499447285.py(30)<cell line: 5>()
     28         for timestep in path:
     29             env.world.agents = timestep
---> 30             assert False
     31             num_subgraph += 1
     32         num_subgraphs.append(num_subgraphs/len(path))



ipdb>  exit()
